# Analysis

This notebook is for extracting the bulk of the results for the project.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import networkx as nx
import stim
import sinter
import model

import matplotlib.pyplot as plt

In [ ]:
# Edit matplotlib parameters
plt.rcParams.update(
    {
        "text.usetex": True,  # WARNING: make false if no TeX compiler installed
        "font.family": "Serif",
        "figure.dpi": 300,
    }
)

## $\text{Hamming}(7,4)$

In [ ]:
clist_hamming_7_4 = ["B", "C", 0, 1, 5, 6, "B", "B", "C", 1, 2, 4, 5, "B", "B", "B", "C", 3, 4, 5, 6, "B"]
clist_rep_3 = ["B", "C", 0, 1, "B", "C", 1, 2, "B"]
pos_rep_3_rep_3 = [(x, 5 - y - 1) for y in range(5) for x in range(5)]
pos_rep_3_hamming_7_4 = [(x, 5 - y - 1) for y in range(5) for x in range(10)]

In [ ]:
hp_code_rep_3_rep_3 = model.StabilizerModel(
    "hypergraph_product_code:z_memory",
    clist1=clist_rep_3,
    clist2=clist_rep_3,
    pos=pos_rep_3_rep_3,
    rounds=2,
)

hp_code_rep_3_rep_3_tripartite = model.StabilizerModel(
    "hypergraph_product_code:z_memory",
    clist1=clist_rep_3,
    clist2=clist_rep_3,
    pos="spring",
    rounds=2,
)

In [ ]:

hp_code_rep_3_hamming_7_4 = model.StabilizerModel(
    "hypergraph_product_code:x_memory",
    clist1=clist_rep_3,
    clist2=clist_hamming_7_4,
    pos=pos_rep_3_hamming_7_4,
    rounds=2,
)